In [1]:
import pandas as pd
import gspread
import copy
from oauth2client.service_account import ServiceAccountCredentials

# glibber glabber that I copied and pasted
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('practice-log-444917-d375b6889398.json', scope)
gc = gspread.authorize(credentials)

# spreadsheet_key = '1AgQWFFgXEYRAtokWU8NNDdGKy5R7DYBoPMFukrRtMiA' # This is the test sheet
spreadsheet_key = '13CdEbddbIDDHdCVU9IvpHQzSB4lJroJ36Riufl5uvTk' # This is the real sheet
book = gc.open_by_key(spreadsheet_key)

In [35]:
# Studio in order of grade then alphabetical by last name
studio = ['Jaime Cardenas', 'Anna Desfore', 'Matt Schneider', 'Tanner Day', 'Trevor Maliborski', 'Adina Shaikh', 'Dylan Chapman', 'Joshua Dieter',
          'Van Maxwell', 'Cadie Strahota', 'Nate Dreyfus', 'Ravlin Frantz', 'Ellis Goeden', 'Kendal Schreiber', 'Alex Seitler',
          'Cooper Atkinson', 'Aaron Dieter', 'Ryan Gomez', 'Annabelle Nault', 'Marley Van Domelan','Eleanor Peska', 'Aidan Lishner', 'Pete Navratil', 'Nick Ruplinger']

In [36]:
# database of names and nicknames

names = {}
for full_name in studio:
    first_name = full_name.split(' ')[0].lower()
    names[full_name] = [first_name]

names['Cooper Atkinson'].append('coop')
names['Ravlin Frantz'].append('rav')
names['Joshua Dieter'].append('josh')

In [37]:
# grade by name

name_grade = {}

# DMA
for name in ['Jaime Cardenas', 'Anna Desfore', 'Matt Schneider']:
    name_grade[name] = 'DMA'

# Master
for name in ['Tanner Day']:
    name_grade[name] = 'Masters'

# Super Senior
for name in ['Trevor Maliborski', 'Adina Shaikh']:
    name_grade[name] = 'Super Senior'

# Senior
for name in ['Dylan Chapman', 'Joshua Dieter', 'Van Maxwell', 'Cadie Strahota']:
    name_grade[name] = 'Senior'
    
# Junior
for name in ['Nate Dreyfus', 'Ravlin Frantz', 'Ellis Goeden', 'Kendal Schreiber', 'Alex Seitler']:
    name_grade[name] = 'Junior'

# Sophomore
for name in ['Cooper Atkinson', 'Aaron Dieter', 'Ryan Gomez', 'Annabelle Nault', 'Marley Van Domelan', 'Eleanor Peska']:
    name_grade[name] = 'Sophomore'

# Freshman
for name in ['Aidan Lishner', 'Pete Navratil', 'Nick Ruplinger']:
    name_grade[name] = 'Freshman'

In [38]:
timeframes = ['7:00 AM','7:15 AM','7:30 AM','7:45 AM','8:00 AM','8:15 AM','8:30 AM','8:45 AM','9:00 AM','9:15 AM','9:30 AM','9:45 AM', 
'10:00 AM','10:15 AM','10:30 AM','10:45 AM','11:00 AM','11:15 AM','11:30 AM','11:45 AM','12:00 PM','12:15 PM','12:30 PM','12:45 PM','1:00 PM', \
'1:15 PM','1:30 PM','1:45 PM','2:00 PM','2:15 PM','2:30 PM','2:45 PM','3:00 PM','3:15 PM','3:30 PM','3:45 PM','4:00 PM','4:15 PM','4:30 PM','4:45 PM', \
'5:00 PM','5:15 PM','5:30 PM','5:45 PM','6:00 PM','6:15 PM','6:30 PM','6:45 PM','7:00 PM','7:15 PM','7:30 PM','7:45 PM','8:00 PM','8:15 PM','8:30 PM', \
'8:45 PM','9:00 PM','9:15 PM','9:30 PM','9:45 PM','10:00 PM','10:15 PM','10:30 PM','10:45 PM']

In [39]:
# function to determine if a name is in the studio, returns full name or name that was inputted
def name_grabber(cell):
    full_name = None
    for names_tuple in names.items():
        for name in names_tuple[1]:
            if name == cell or name in cell:
                full_name = names_tuple[0]
        if full_name != None:
            break
    if full_name == None:
        return cell
    else:
        return full_name

In [40]:
# Grab that data

# Get different sheets
rooms = ["Studio", "1401", "1407", "1409", "1413", "1414", "1416", "1417", "1418"]

entries_by_person = {}
entries_by_timeframe = {}
entries_by_person_timeframe = {}
entries_by_room = {}
entries_by_room_wo_lessons = {}
other_entries = []
for room in rooms:
    for week in range(1,3):
        room_week = (room + " (" + str(week) + ")")
        worksheet = book.worksheet(room_week)
        table = worksheet.get_all_values()
        
        # Format Table in Pandas
        # Time      Mon   Tue ...
        # 7:00 AM   Josh   '' ...
        # 7:15 AM   Josh   '' ...
        # ...
        df = pd.DataFrame(table[3:])
        df.columns = ['Time', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        
        # Modify dataframe so it fills in gaps in current dataframe
        for day in df:
            for idx in range(len(df[day])):
                cell = df[day].iloc[idx]
                if cell != '':
                    diff = 1
                    for idx2 in range(1,9):
                        if (idx+idx2) > (len(df[day])-1):   # max bounds case
                            break
                        next_cell = df[day].iloc[idx+idx2]
                        if (next_cell != cell) and (next_cell != ''):
                            break
                        if next_cell == cell:
                            diff = idx2
                            break
                    for i in range(diff):
                        df.loc[i+idx,day] = cell
                        
        # Indexes aren't needed anymore so setting 'Time' as index
        df = df.set_index('Time').rename_axis(None)
        
        # total entries by person
        # and entries by person by timeframe
        for day in df:
            for timeframe_cell in df[day].items():
                timeframe = timeframe_cell[0]
                cell = timeframe_cell[1]
                lcell = cell.lower().strip()
                if lcell == '':   # if cell is empty, skip to next cell
                    continue
                full_name = name_grabber(lcell) # grabs full name

                # other entries
                if full_name == lcell:
                    if full_name not in other_entries:
                        other_entries.append(full_name)

                # entries by person
                if full_name not in entries_by_person:
                    entries_by_person[full_name] = 0
                    entries_by_person_timeframe[full_name] = {}
                entries_by_person[full_name] += 1

                # entries by person by timeframe
                if timeframe not in entries_by_person_timeframe[full_name]:
                    entries_by_person_timeframe[full_name][timeframe] = 0
                entries_by_person_timeframe[full_name][timeframe] += 1
                
                
        # total entries by room
        if room not in entries_by_room:
            entries_by_room[room] = 0
        for day in df:
            for cell in df[day]:
                if cell != '':
                    entries_by_room[room] += 1

        # total entries by room without lessons
        if room not in entries_by_room_wo_lessons:
            entries_by_room_wo_lessons[room] = 0
        for day in df:
            for cell in df[day]:
                if cell != '' and cell != 'LESSONS':
                    entries_by_room_wo_lessons[room] += 1

        # counting how many entries are in each 15min timeframe
        for timeframe in timeframes:
            timeframe_series = df.loc[timeframe]
            for cell in timeframe_series:
                if cell != '':
                    if timeframe not in entries_by_timeframe:
                        entries_by_timeframe[timeframe] = 0
                    entries_by_timeframe[timeframe] += 1
                    
        
# sorted_entries_by_person = dict(sorted(entries_by_person.items(), key=lambda ele: ele[1], reverse = True))
# sorted_entries_by_room = dict(sorted(entries_by_room.items(), key=lambda ele: ele[1], reverse = True))
# sorted_entries_by_room_wo_lessons = dict(sorted(entries_by_room_wo_lessons.items(), key=lambda ele: ele[1], reverse = True))

In [41]:
sorted_entries_by_timeframe = dict(sorted(entries_by_timeframe.items(), key=lambda ele: ele[1], reverse = True))
sorted_entries_by_timeframe

{'11:30 AM': 37,
 '11:45 AM': 37,
 '3:00 PM': 37,
 '10:30 AM': 36,
 '11:15 AM': 36,
 '1:00 PM': 36,
 '1:15 PM': 36,
 '1:45 PM': 36,
 '2:00 PM': 36,
 '3:30 PM': 36,
 '10:00 AM': 35,
 '11:00 AM': 35,
 '10:45 AM': 35,
 '12:00 PM': 35,
 '12:45 PM': 35,
 '12:15 PM': 34,
 '12:30 PM': 34,
 '2:15 PM': 34,
 '3:15 PM': 34,
 '10:15 AM': 33,
 '1:30 PM': 33,
 '3:45 PM': 32,
 '2:30 PM': 31,
 '2:45 PM': 31,
 '4:00 PM': 30,
 '4:15 PM': 28,
 '4:45 PM': 27,
 '9:30 AM': 25,
 '4:30 PM': 25,
 '9:15 AM': 22,
 '5:00 PM': 22,
 '9:45 AM': 21,
 '9:00 AM': 19,
 '5:15 PM': 18,
 '5:45 PM': 16,
 '8:30 AM': 15,
 '5:30 PM': 15,
 '6:00 PM': 15,
 '6:15 PM': 15,
 '6:45 PM': 13,
 '8:45 AM': 12,
 '6:30 PM': 12,
 '7:00 PM': 11,
 '7:15 PM': 10,
 '7:45 PM': 10,
 '8:15 PM': 10,
 '8:30 PM': 10,
 '8:15 AM': 9,
 '7:30 PM': 9,
 '8:00 PM': 9,
 '8:45 PM': 9,
 '9:00 PM': 7,
 '8:00 AM': 7,
 '7:45 AM': 5,
 '9:15 PM': 4,
 '7:30 AM': 4,
 '9:30 PM': 3,
 '9:45 PM': 3,
 '7:15 AM': 3,
 '7:00 AM': 2,
 '10:00 PM': 1,
 '10:15 PM': 1,
 '10:45 P

In [50]:
# make separate lists for names in studio and other

entries_by_studio = copy.deepcopy(entries_by_person)
entries_by_other = {}
for person_entries in entries_by_person.items():
    if person_entries[0] in other_entries:
        entries_by_other[person_entries[0]] = entries_by_studio.pop(person_entries[0])

entries_by_other

{'ganner': 1,
 'lessons': 89,
 'rep': 6,
 'lesson': 1,
 'aed': 11,
 'tom': 12,
 'maey': 1,
 'vibe out': 26,
 'vibe back': 1,
 'marimba out': 44,
 'msrley': 2}

In [27]:
# # entries_to_time(dict) function to convert 15 min entries to _ hr __ min

# def entries_to_time(dict):
#     converted_dict = {}
#     for item1_item2 in dict.items():
#         item1 = item1_item2[0]
#         item2 = item1_item2[1]
#         hours = (item2*15)/60
#         hours_mins = str(hours).split('.')
#         if int(hours_mins[0]) == 0:
#             converted_dict[item1] = str(int(float('.'+hours_mins[1])*60)) + 'mins'
#         else: 
#             converted_dict[item1] = str(hours_mins[0]) + 'hrs ' + str(int(float('.'+hours_mins[1])*60)) + 'mins'
#     return converted_dict

In [45]:
# get the stats worksheet
stats = book.worksheet('Stats')

[['UW-Madison', 'Percussion Practice Room Statistics', '', '', '', '', '', ''], ['', 'As of January 20', '', '', 'Instruments: Python 3.13.0, gspread', '', '', ''], ['Individual Stats', '', '', '', 'Room Stats', '', '', ''], ['Studio Members', '', 'Other', '', 'Lessons Included', '', 'Lessons Excluded', ''], ['Joshua Dieter', '8hrs 45mins', 'lessons', '23hrs 30mins', 'Studio', '36hrs 0mins', 'Studio', '23hrs 0mins'], ['Alex Seitler', '7hrs 30mins', 'arcomusical', '15mins', '1401', '22hrs 45mins', '1401', '22hrs 45mins'], ['Dylan Chapman', '15hrs 30mins', 'arco', '3hrs 45mins', '1407', '7hrs 15mins', '1407', '7hrs 15mins'], ['Ravlin Frantz', '8hrs 30mins', 'ted', '2hrs 0mins', '1409', '58hrs 30mins', '1409', '48hrs 0mins'], ['Aaron Dieter', '7hrs 30mins', 'lesso', '15mins', '1413', '30hrs 0mins', '1413', '30hrs 0mins'], ['Ellis Goeden', '14hrs 15mins', 'a dina', '30mins', '1414', '24hrs 15mins', '1414', '24hrs 15mins'], ['Van Maxwell', '9hrs 45mins', 'marlwy', '15mins', '1416', '5hrs 45

In [29]:
# # individual stats
# time_by_studio = entries_to_time(entries_by_studio)
# time_by_other = entries_to_time(entries_by_other)

# # add to leaderboard with studio only values
# indiv_idx = 5
# for person_time in time_by_studio.items():
#     person = person_time[0]
#     time = person_time[1]
#     stats.update_acell('A'+str(indiv_idx),person)
#     stats.update_acell('B'+str(indiv_idx),time)
#     indiv_idx += 1

# # add to other with other entries
# indiv_idx = 5
# for person_time in time_by_other.items():
#     person = person_time[0]
#     time = person_time[1]
#     stats.update_acell('C'+str(indiv_idx),person)
#     stats.update_acell('D'+str(indiv_idx),time)
#     indiv_idx += 1

In [30]:
# # add room stats

# # first, convert 15 min entries to hr min
# time_by_room = entries_to_time(entries_by_room)
# time_by_room_wo_lessons = entries_to_time(entries_by_room_wo_lessons)

# # with lessons
# room_idx = 5
# for room_time in time_by_room.items():
#     room = room_time[0]
#     time = room_time[1]
#     stats.update_acell('E'+str(room_idx),room)
#     stats.update_acell('F'+str(room_idx),time)
#     room_idx += 1

# # without lessons
# room_idx = 5
# for room_time in time_by_room_wo_lessons.items():
#     room = room_time[0]
#     time = room_time[1]
#     stats.update_acell('G'+str(room_idx),room)
#     stats.update_acell('H'+str(room_idx),time)
#     room_idx += 1

In [46]:
# dataframe with all the stats sorted in alphabetical order by name

# take list of names and entry amounts (excluding other, also unsorted) and convert to dataframe, add other onto the end so its more organized

# combine studio and other entries
entries_by_combined = entries_by_studio|entries_by_other
# s_entries_by_combined = dict(sorted(entries_by_combined.items(), key=lambda ele: ele[0], reverse = False))

name_entries_df = pd.DataFrame.from_dict(entries_by_combined, orient="index").reset_index()
name_entries_df.columns = ['Name','Entries']

# add grade level
name_grade_df = pd.DataFrame.from_dict(name_grade, orient='index').reset_index()
name_grade_df.columns = ['Name','Grade']

# merge entries and grade level
perc_stats = pd.merge(name_grade_df, name_entries_df, on='Name',how='outer')

# fill NaN
perc_stats.fillna({'Grade': 'Unknown', 'Entries': 0}, inplace=True)

# conversion function
def entries_to_time(row):
    hours = row['Entries']*15/60
    hours_mins = str(hours).split('.')
    if int(hours_mins[0]) == 0:
        return str(int(float('.'+hours_mins[1])*60)) + 'mins'
    else: 
        return str(hours_mins[0]) + 'hrs ' + str(int(float('.'+hours_mins[1])*60)) + 'mins'

# Time Practiced
perc_stats['Time Practiced'] = perc_stats.apply(entries_to_time, axis=1)

def entries_to_time_p_day(row):
    hours = row['Entries']*15/60/14  # divide by the 7 days in the week
    hours_mins = str(hours).split('.')
    if int(hours_mins[0]) == 0:
        return str(int(float('.'+hours_mins[1])*60)) + 'mins'
    else: 
        return str(hours_mins[0]) + 'hrs ' + str(int(float('.'+hours_mins[1])*60)) + 'mins'

# Time/Day
perc_stats['Time per Day (14)'] = perc_stats.apply(entries_to_time_p_day, axis=1)

# fav timeframe function
def fav_timeframe(row):
    for tuple in entries_by_person_timeframe.items():
        name = tuple[0]
        dict = tuple[1]
        if name == row['Name']:
            return max(dict, key=dict.get)
    return 'N/A'

# Favorite Practice Timeframe
perc_stats['Favorite Practice Timeframe'] = perc_stats.apply(fav_timeframe, axis=1)

sorted_perc_stats = perc_stats.sort_values('Entries', ascending = False)

sorted_perc_stats[sorted_perc_stats['Grade'] == 'Junior']

sorted_perc_stats.groupby('Grade').sum('Entries')

entries_by_grade = sorted_perc_stats.groupby('Grade').agg(
    Count=('Entries', 'count'),
    Entries=('Entries', 'sum')
)

entries_by_grade['avg'] = entries_by_grade['Entries']/entries_by_grade['Count']
entries_by_grade

perc_stats

,Name,Grade,Entries,Time Practiced,Time per Day (14),Favorite Practice Timeframe
0,Aaron Dieter,Sophomore,70.0,17hrs 30mins,1hrs 15mins,3:00 PM
1,Adina Shaikh,Super Senior,46.0,11hrs 30mins,49mins,4:30 PM
2,Aidan Lishner,Freshman,30.0,7hrs 30mins,32mins,6:45 PM
3,Alex Seitler,Junior,54.0,13hrs 30mins,57mins,8:30 AM
4,Anna Desfore,DMA,109.0,27hrs 15mins,1hrs 56mins,5:00 PM
5,Annabelle Nault,Sophomore,0.0,0mins,0mins,N/A
6,Cadie Strahota,Senior,39.0,9hrs 45mins,41mins,12:00 PM
7,Cooper Atkinson,Sophomore,48.0,12hrs 0mins,51mins,12:45 PM
8,Dylan Chapman,Senior,40.0,10hrs 0mins,42mins,12:30 PM
9,Eleanor Peska,Sophomore,55.0,13hrs 45mins,58mins,11:00 AM


In [47]:
sorted_perc_stats[sorted_perc_stats['Grade'] == 'Freshman']

,Name,Grade,Entries,Time Practiced,Time per Day (14),Favorite Practice Timeframe
18,Pete Navratil,Freshman,78.0,19hrs 30mins,1hrs 23mins,1:15 PM
17,Nick Ruplinger,Freshman,44.0,11hrs 0mins,47mins,11:30 AM
2,Aidan Lishner,Freshman,30.0,7hrs 30mins,32mins,6:45 PM


In [49]:
sorted_perc_stats[sorted_perc_stats["Grade"] != "Unknown"]

,Name,Grade,Entries,Time Practiced,Time per Day (14),Favorite Practice Timeframe
10,Ellis Goeden,Junior,154.0,38hrs 30mins,2hrs 45mins,11:00 AM
4,Anna Desfore,DMA,109.0,27hrs 15mins,1hrs 56mins,5:00 PM
18,Pete Navratil,Freshman,78.0,19hrs 30mins,1hrs 23mins,1:15 PM
0,Aaron Dieter,Sophomore,70.0,17hrs 30mins,1hrs 15mins,3:00 PM
14,Marley Van Domelan,Sophomore,70.0,17hrs 30mins,1hrs 15mins,11:00 AM
9,Eleanor Peska,Sophomore,55.0,13hrs 45mins,58mins,11:00 AM
3,Alex Seitler,Junior,54.0,13hrs 30mins,57mins,8:30 AM
13,Kendal Schreiber,Junior,50.0,12hrs 30mins,53mins,4:30 PM
19,Ravlin Frantz,Junior,50.0,12hrs 30mins,53mins,3:45 PM
7,Cooper Atkinson,Sophomore,48.0,12hrs 0mins,51mins,12:45 PM
